In [1]:
%matplotlib inline

import json
import ast

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
from typing import List

from jass.logs.game_log_entry import GameLogEntry
from jass.game.game_state import GameState
from jass.game.game_state_util import state_from_complete_game, observation_from_state
from jass.game.game_util import convert_int_encoded_cards_to_str_encoded, convert_one_hot_encoded_cards_to_str_encoded_list

from jass.game.rule_schieber import RuleSchieber

game_rule = RuleSchieber()
path_to_data = Path('../data')

# Card Selection with Deep Neural Network (Keras)

Data originally from Swisslos Jass Platform is available
- under Data/games on Illias,
- and on the /exchange/dl4g/data/games folder on the GPU cluster
- Each data file contains 100000 games
- Each line is a complete game in json format that can be read by


GameLogEntry.from_json() to get an entry
- An entry consists of:
- The game (as GameState for a complete game)
- The date
- The player ids on the server, id=0 for anonymous play

## Data Analysis

Here is an example GameEntry datastructure

In [ ]:
GameLogEntry.from_json(
    {
        "game": {
            "version": "V0.2",
            "trump": 1,
            "dealer": 2,
            "currentPlayer": -1,
            "forehand": 0,
            "tricks": [
                {
                    "cards": ["HA", "H9", "H10", "D10"],
                    "points": 45,
                    "win": 0,
                    "first": 1,
                },
                {
                    "cards": ["CJ", "CA", "C7", "C10"],
                    "points": 23,
                    "win": 3,
                    "first": 0,
                },
                {
                    "cards": ["S6", "S10", "SQ", "S9"],
                    "points": 13,
                    "win": 1,
                    "first": 3,
                },
                {"cards": ["SA", "C6", "H7", "S7"], "points": 11, "win": 3, "first": 1},
                {"cards": ["SK", "SJ", "S8", "C8"], "points": 6, "win": 3, "first": 3},
                {"cards": ["D7", "DJ", "HQ", "DA"], "points": 16, "win": 1, "first": 3},
                {"cards": ["D9", "DK", "HK", "DQ"], "points": 11, "win": 3, "first": 1},
                {"cards": ["HJ", "C9", "H6", "D6"], "points": 20, "win": 3, "first": 3},
                {"cards": ["H8", "CQ", "D8", "CK"], "points": 12, "win": 3, "first": 3},
            ],
            "player": [{"hand": []}, {"hand": []}, {"hand": []}, {"hand": []}],
            "jassTyp": "SCHIEBER",
        },
        "date": "13.10.17 22:31:05",
        "player_ids": [58663, 21516, 41630, 70654],
    }
)

In order to load a file of games we need to iterate over the lines and convert them to GameLogEntry objects

In [ ]:
# TODO: when model is defined and ready for training add all files to the training data

game_entries = []
with open(path_to_data / 'games/jass_game_0001.txt') as f:
    for line in f:
        game_entries.append(GameLogEntry.from_json(json.loads(line)))
    
game_entries = np.array(game_entries)
print("%d bytes" % (game_entries.size * game_entries.itemsize))


### Specific GameState from Complete GameState

In order to retriev a specific GameState from a complete GameState we can use the `state_from_complete_game` function.

In [ ]:
state_from_complete_game(game_entries[0].game, 5).to_json()

## Analyse Player Stats

In [ ]:
player_stats = pd.read_json(path_to_data / 'stat' / 'player_all_stat.json')
player_stats.head()

In [ ]:
sns.boxplot(player_stats['mean']) 

In [7]:
player_stats = player_stats[player_stats['mean'] > player_stats['mean'].mean()]

In [8]:
player_stats = player_stats[player_stats['nr'] > 100]

In [ ]:
sns.boxplot(player_stats['mean']) 

In [10]:
# collect player ids and build dataset from their views
player_ids = player_stats['id'].to_numpy()

## Dataset Structure for Model Training

### Simple
Features: (2x36 = 72)
- cards played in current round (9x9 = 36)
- valid cards (9x9 = 36)

Target: (9x9 = 36 -> only select one! / or probability)
- choose one of the valid cards in own hand
- one-hot-encoding of all cards
- select only cards that are available
- select only cards that are valid

### More Useful Information
Features: (4x36 = 144)
- already played cards (9x9 = 36)
- cards played in current round (9x9 = 36)
- cards in hand (9x9 = 36)
- valid cards (9x9 = 36)

Target: (9x9 = 36 -> only select one! / or probability)
- choose one of the valid cards in own hand
- one-hot-encoding of all cards
- select only cards that are available
- select only cards that are valid

# Simple Card Play Model

Remove all GameLogs that do not have a user in the calculated userid list

In [ ]:
good_games: List[GameLogEntry] = [game for game in game_entries if any(player in player_ids for player in game.player_ids)]

len(good_games)

In [ ]:
data = pd.DataFrame({'trump': 'TPTP', 'played_cards': [['TP']], 'valid_cards': [['TP', 'TP', 'TP', 'TP', 'TP', 'TP', 'TP', 'TP', 'TP']], 'selected_card': ['TP']})

data.head()

In [ ]:
# Print iterations progress
def printProgressBar(iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()
        
def collect_data_from_game_logs():
    max_iterations = len(good_games)
    printProgressBar(0, max_iterations, prefix = 'Progress:', suffix = 'Complete', length = 50)
    
    data_rows = []  # Store rows here and add them to DataFrame in bulk at the end
    
    for idx, game_log in enumerate(good_games):
        if idx % 10 == 0:  # Update the progress bar less frequently
            printProgressBar(idx + 1, max_iterations, prefix='Progress:', suffix='Complete', length=50)
                
        for player_index, player in enumerate(game_log.player_ids):
            if player not in player_ids:
                continue
                    
            for cards_played in range(36):  # Iterate through all possible states
                state = state_from_complete_game(game_log.game, cards_played)
                if state.player != player_index:  # Check if it's the specified player's turn
                    continue
                    
                played_cards = []

                # Iterate through the tricks to collect played cards
                for trick in state.tricks:
                    for card in trick:
                        if card != -1:  # Only consider valid cards
                            played_cards.append(card)

                # Convert to string representation if needed
                played_cards_str = convert_int_encoded_cards_to_str_encoded(played_cards)
                
                valid_cards = game_rule.get_valid_cards_from_state(state)
                valid_cards_str = convert_one_hot_encoded_cards_to_str_encoded_list(valid_cards)
                
                # evaluate the played card by the player from the next following state
                selected_card = -1
                next_state = None
                if cards_played == 35:
                    next_state = game_log.game
                    selected_card = next_state.tricks[-1, -1]
                else:
                    next_state = state_from_complete_game(game_log.game, cards_played + 1)
                    next_state_trick = [card for card in next_state.current_trick if card != -1]
                    if len(next_state_trick) != 0:
                        selected_card = next_state_trick[-1]
                    else:
                        next_state_trick = next_state.tricks[next_state.nr_tricks - 1, :]
                        selected_card = next_state_trick[-1]
                    
                selected_card_str = convert_int_encoded_cards_to_str_encoded([selected_card])
                
                # Add the game state as a row
                data_rows.append([
                    game_log.game.trump, played_cards_str, valid_cards_str, selected_card_str[0]
                ])
                
        if idx % 100 == 0:
            global data
            data = pd.concat([data, pd.DataFrame(data_rows, columns=['trump', 'played_cards', 'valid_cards', 'selected_card'])], ignore_index=True)
            data_rows = []
            assert len(data_rows) == 0
                
collect_data_from_game_logs()

In [ ]:
data.count()

In [ ]:
data.head()

In [81]:
data.trump = data.trump.astype('category')
data.trump = data.trump.cat.rename_categories({0: 'DIAMONDS', 1: 'HEARTS', 2: 'SPADES', 3:'CLUBS',
                                  4: 'OBE_ABE', 5: 'UNE_UFE', 6: 'PUSH', 10: 'PUSH'})

In [ ]:
data.trump.unique()

In [83]:
data.to_csv(path_to_data / 'card_prediction' / 'card_prediciton_0001.csv', index=False)